In [2]:
!unzip images/pakistani_fashion_dataset.zip -d extracted_files

Archive:  images/pakistani_fashion_dataset.zip
   creating: extracted_files/pakistani_fashion_dataset/
  inflating: extracted_files/pakistani_fashion_dataset/.DS_Store  
  inflating: extracted_files/__MACOSX/pakistani_fashion_dataset/._.DS_Store  
  inflating: extracted_files/pakistani_fashion_dataset/.gitattributes  
   creating: extracted_files/pakistani_fashion_dataset/train/
   creating: extracted_files/pakistani_fashion_dataset/.git/
  inflating: extracted_files/pakistani_fashion_dataset/train/9733.jpg  
  inflating: extracted_files/pakistani_fashion_dataset/train/63.jpg  
  inflating: extracted_files/pakistani_fashion_dataset/train/6400.jpg  
  inflating: extracted_files/pakistani_fashion_dataset/train/823.jpg  
  inflating: extracted_files/pakistani_fashion_dataset/train/4217.jpg  
  inflating: extracted_files/pakistani_fashion_dataset/train/3578.jpg  
  inflating: extracted_files/pakistani_fashion_dataset/train/2666.jpg  
  inflating: extracted_files/pakistani_fashion_dataset/t

In [8]:
!pip install torch transformers pillow pandas peft numpy==1.26.3




In [6]:
import subprocess

# URL of the existing Space repository
repo_url = "https://huggingface.co/spaces/Quan9029/Fashion-Image-Captioning-using-BLIP-2"

# Clone the repository
subprocess.run(["git", "clone", repo_url])

Cloning into 'Fashion-Image-Captioning-using-BLIP-2'...
Filtering content: 100% (11/11), 7.23 GiB | 117.67 MiB/s, done.


CompletedProcess(args=['git', 'clone', 'https://huggingface.co/spaces/Quan9029/Fashion-Image-Captioning-using-BLIP-2'], returncode=0)

In [13]:
from transformers import Blip2ForConditionalGeneration, Blip2Processor
from peft import PeftModel
from PIL import Image
import pandas as pd
import os
import torch

# Check for GPU availability and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


preprocess_ckp = "Salesforce/blip2-opt-2.7b"
base_model_ckp = "./Fashion-Image-Captioning-using-BLIP-2/model/blip2-opt-2.7b-fp16-sharded"
peft_model_ckp = "./Fashion-Image-Captioning-using-BLIP-2/model/blip2_peft"

# Initialize processor and model
processor = Blip2Processor.from_pretrained(preprocess_ckp, revision="51572668da0eb669e01a189dc22abe6088589a24")
model = Blip2ForConditionalGeneration.from_pretrained(base_model_ckp, revision="51572668da0eb669e01a189dc22abe6088589a24")
model = PeftModel.from_pretrained(model, peft_model_ckp, revision="51572668da0eb669e01a189dc22abe6088589a24")

model.to(device)




Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

PeftModel(
  (base_model): LoraModel(
    (model): Blip2ForConditionalGeneration(
      (vision_model): Blip2VisionModel(
        (embeddings): Blip2VisionEmbeddings(
          (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
        )
        (encoder): Blip2Encoder(
          (layers): ModuleList(
            (0-38): 39 x Blip2EncoderLayer(
              (self_attn): Blip2Attention(
                (dropout): Dropout(p=0.0, inplace=False)
                (qkv): Linear(in_features=1408, out_features=4224, bias=True)
                (projection): Linear(in_features=1408, out_features=1408, bias=True)
              )
              (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
              (mlp): Blip2MLP(
                (activation_fn): GELUActivation()
                (fc1): Linear(in_features=1408, out_features=6144, bias=True)
                (fc2): Linear(in_features=6144, out_features=1408, bias=True)
              )
              

In [15]:
# image_dir = "extracted_files/pakistani_fashion_dataset/train"
# output_csv = "metadata.csv"
# data = []

# for img_file in os.listdir(image_dir):
#     if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
#         image_path = os.path.join(image_dir, img_file)
#         image = Image.open(image_path).convert("RGB")
#         inputs = processor(images=image, return_tensors="pt")
#         pixel_values = inputs.pixel_values
#         generated_ids = model.generate(pixel_values=pixel_values, max_length=10)
#         generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
#         data.append({"image": image_path, "text": generated_caption})
#         print(f"Processed {img_file}: {generated_caption}")

# # Save to CSV
# df = pd.DataFrame(data)
# df.to_csv(output_csv, index=False)

image_dir = "extracted_files/pakistani_fashion_dataset/train"
output_csv = "metadata.csv"
data = []

for img_file in os.listdir(image_dir):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_dir, img_file)
        image = Image.open(image_path).convert("RGB")
        # Preprocess the image and move the inputs to the same device as the model
        inputs = processor(images=image, return_tensors="pt").to(device)
        # Generate captions
        generated_ids = model.generate(**inputs, max_length=10)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        data.append({"image": image_path, "text": generated_caption})
        print(f"Processed {img_file}: {generated_caption}")

# Save the results to a CSV file
df = pd.DataFrame(data)
df.to_csv(output_csv, index=False)

Processed 6022.jpg: black pink embroidered dupatta
Processed 1332.jpg: green kashmiri emerald earring
Processed 3737.jpg: green kamila long sleeve tunic dress
Processed 617.jpg: navy blue embroidered chiffon suit
Processed 4200.jpg: gold black onyx ring
Processed 2541.jpg: black gold embroidered salwar suit
Processed 1649.jpg: white chloe turtleneck sweater dress
Processed 3191.jpg: blue multi embroidered pajamas
Processed 5376.jpg: brown leather khan flat
Processed 2739.jpg: cream embroidered lace tunic
Processed 5792.jpg: teal gold embroidered kurta set
Processed 4384.jpg: black kurta pants
Processed 9942.jpg: beige multi embroidered salwar suit
Processed 9675.jpg: pink floral print cotton pajamas pants
Processed 9768.jpg: red multi embroidered salwar suit
Processed 249.jpg: burgundy velvet salwar suit
Processed 167.jpg: white arabic logo tee
Processed 5015.jpg: white note powder compact foundation
Processed 3123.jpg: blue multi kamla lawn dress
Processed 9054.jpg: coral pink embroid